<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Steam Web Scraping</span>  </h1>

<img src="Screenshot 2022-09-12 at 10.12.59 PM.png">


<b>Objective</b>: Steam is the ultimate destination for playing, discussing, and creating games. 

* We want to scrape the details of top 5 "New and Trending" games.
* For every game, we will find out the number of positive reviews, name of developer and publisher.
* Find out the system requirements for each game.
* Scrape 10 reviews for each game.

In [1]:
import requests                 # Retrieving data from the website
from bs4 import BeautifulSoup          # Used for performing Web Scraping

from scrapy import Selector            # module for webscraping and crawling

import re                       ## Importing Regex for string matching
import time                   ## Timing the Program

In [2]:
# Website we are going to scrape data from
base_site = 'https://store.steampowered.com/explore/new/'

In [3]:
response = requests.get(base_site)       ## Requesting the base_site for the code
response

<Response [200]>

In [4]:
html = response.content          ## Getting all the content (HTML code) from the response

In [5]:
sel = Selector(text = html)

In [6]:
soup = BeautifulSoup(html, 'lxml' )    ## lxml parser
# soup                                 ## prints entire html code for the base_site

<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">New & Trending Games</span>  </h1>

In [7]:
# All top games have their respective names saved in div with 'class' = 'tab_item_name'
divs = soup.find_all('div', {'class': 'tab_item_name'})  
divs[:5]       # Only first element                      ## First five elements

[<div class="tab_item_name">Trombone Champ</div>,
 <div class="tab_item_name">hololive ERROR</div>,
 <div class="tab_item_name">Absolute Tactics: Daughters of Mercy</div>,
 <div class="tab_item_name">Idle Skilling</div>,
 <div class="tab_item_name">Foretales</div>]

In [8]:
game_name = [div.string for div in divs][:17]
game_name[:5]                                   ## The output gives us the name of all top 5 games

['Trombone Champ',
 'hololive ERROR',
 'Absolute Tactics: Daughters of Mercy',
 'Idle Skilling',
 'Foretales']

Now, we need to get hyperlink attached to each game to get the full game information. We can make a function for that, but before that we will scrape the information of one game and then simply define it as a function to repeat the process for all the games.

In [9]:
urls = []                     ## Urls for all the games

for i in range(1, len(game_name) + 1):             ## 10 games for now
    urls_5 = sel.xpath(f'//*[@id="tab_newreleases_content"]/a[{i}]/@href').extract()
    urls.append(urls_5[0])
    
#urls  --- All the urls for the games
#len(urls)

In [10]:
tags = []                 ## All the unique ids for games 

for i in urls:
    tags.append(re.findall(r'\d+', i)[0])

int_tags = [int(j) for j in tags]
## int_tags          -- unique tags of all the games  
## len(int_tags)


<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Game Information</span>  </h1>

In [11]:
num_reviews = []                ## Stores number of reviews for all games
rel_date = []                   ## Stores Release Dates of all games
dev_names = []                  ## Stores Developer Name/Names for all games
pub_names = []                  ## Stores Publisher Name/Names for all games
min_reqs = []                   ## Stores System Requirements for all games
all_reviews = []                ## Stores Reviews for all games

for u in range(0, len(urls)):

    website =  urls[u]                                    #'https://store.steampowered.com/app/1556790/Isonzo/'
    response = requests.get(website)       ## Requesting the base_site for the code
    # response

    html = response.content          ## Getting all the content (HTML code) from the response

    ## Making the soup
    soup = BeautifulSoup(html, 'lxml' )    ## lxml parser
    #soup                                 ## prints entire html code for the website
    
    start_time = time.time()        ## Loop Start
    
    ########################### Number of Positive Reviews ######################################
    # All top games have their respective names saved in div with 'class' = 'tab_item_name'
    spans = soup.find('span', class_ = 'responsive_hidden')  
    spans.get_text(strip = True)       # Removes \r\n\t from the output
    
    no_pos_review = int(spans.get_text(strip = True).strip('()').replace(',', ''))    ## removes () from the output, convert to int
    no_pos_review                                                                     ## Replace removes ','
    num_reviews.append(no_pos_review)             ## Appending to num_reviews for no. of reviews for all games
    
    
    ########################### Release Date ######################################
    
    divs = soup.find('div', {'class': 'date'})
    release_date = divs.get_text(strip = True)
    release_date
    rel_date.append(release_date)
    
    ########################### Developer Names ######################################
    
    divs = soup.find('div', {'id': 'developers_list'})
    
    dev_name = divs.get_text(strip = True)
    dev_name
    dev_names.append(dev_name)
    
    ########################### Publisher Names ######################################
    divs = soup.find_all('div', {'class': 'dev_row'})
    
    pub_name = divs[1].get_text(strip = True).replace('Publisher:', '')   # Removing 'Publisher:' from result 
    pub_name
    pub_names.append(pub_name)
    
    ########################### System Requirements ######################################
    divs = soup.find('div', {'class': 'game_area_sys_req_leftCol'})
    
    if divs is None:
        divs = soup.find('div', {'class': 'game_area_sys_req_full'})
        min_req = divs.get_text(strip = True).replace('Minimum:', 'Minimum: ').replace('Processor:', ', Processor: ').replace('Memory:', ', Memory: ').replace('Graphics:', ', Graphics: ').replace('Storage:', ', Storage: ').replace('Sound Card:', ', Sound Card: ')
        
        min_reqs.append(min_req)
        
    else:
        min_req = divs.get_text(strip = True).replace('Minimum:', 'Minimum: ').replace('Processor:', ', Processor: ').replace('Memory:', ', Memory: ').replace('Graphics:', ', Graphics: ').replace('Storage:', ', Storage: ').replace('Sound Card:', ', Sound Card: ')
        min_reqs.append(min_req)
    
    
    ########################### Reviews ######################################
    ## Getting reviews

    website = 'https://steamcommunity.com/app/' + str(int_tags[u]) + '/reviews/?browsefilter=toprated&snr=1_5_100010_'
    response = requests.get(website)       ## Requesting the base_site for the code
    # response

    html = response.content          ## Getting all the content (HTML code) from the response

    ## Making the soup
    soup = BeautifulSoup(html, 'lxml' )    ## lxml parser
    #soup  
    
    divs = soup.find_all('div', {'class': 'apphub_CardTextContent'})          
    
    reviews = [div.get_text(strip = True) for div in divs]
    reviews
    all_reviews.append(reviews)
    
    end_time = time.time()                     ## loop ends
    print(f'{game_name[u]} ------> Time Taken: {round(end_time - start_time)} Seconds')
    print('')
    
    
    
    

Trombone Champ ------> Time Taken: 2 Seconds

hololive ERROR ------> Time Taken: 6 Seconds

Absolute Tactics: Daughters of Mercy ------> Time Taken: 2 Seconds

Idle Skilling ------> Time Taken: 1 Seconds

Foretales ------> Time Taken: 1 Seconds

Metal: Hellsinger ------> Time Taken: 2 Seconds

Night At the Gates of Hell ------> Time Taken: 3 Seconds

Cities: Skylines - Plazas & Promenades ------> Time Taken: 2 Seconds

Lost Judgment ------> Time Taken: 2 Seconds

Judgment ------> Time Taken: 2 Seconds

Freedom Planet 2 ------> Time Taken: 2 Seconds

SCP: Secret Files ------> Time Taken: 2 Seconds

Isonzo ------> Time Taken: 1 Seconds

Europa Universalis IV: Lions of the North ------> Time Taken: 4 Seconds

Roadwarden ------> Time Taken: 1 Seconds

Taiji ------> Time Taken: 2 Seconds

Steelrising ------> Time Taken: 2 Seconds



In [12]:
#num_reviews #rel_date #dev_names #pub_names #min_reqs #all_reviews

<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Saving as a DataFrame</span>  </h1>

In [13]:
import pandas as pd                   ## Importing pandas 
pd.set_option('display.max_colwidth', None)    ## Gives the entire view of the DataFrame

In [17]:
games_info = pd.DataFrame()            ## Empty DataFrame

games_info['Name'] = game_name                                 ## Name of the Game
games_info['URL'] = urls
games_info['Number of Positive Reviews'] = num_reviews
games_info['Date of Release'] = rel_date
games_info['Developer/Developers'] = dev_names
games_info['Publisher/Publishers'] = pub_names
games_info['System Requirements'] = min_reqs
games_info['Reviews'] = all_reviews

games_info.set_index('Name', inplace = True)
games_info

URL  \
Name                                                                                                                                                 
Trombone Champ                                                       https://store.steampowered.com/app/1059990/Trombone_Champ/?snr=1_239_new__145   
hololive ERROR                                                       https://store.steampowered.com/app/2062550/hololive_ERROR/?snr=1_239_new__145   
Absolute Tactics: Daughters of Mercy            https://store.steampowered.com/app/1636660/Absolute_Tactics_Daughters_of_Mercy/?snr=1_239_new__145   
Idle Skilling                                                         https://store.steampowered.com/app/1048370/Idle_Skilling/?snr=1_239_new__145   
Foretales                                                                 https://store.steampowered.com/app/1170080/Foretales/?snr=1_239_new__145   
Metal: Hellsinger                                                  https://store.steampowered.com/app/1061910/Metal_Hellsinger/?snr=1_239_new__145   
Night At the Gates of Hell                               https://store.steampowered.com/app/2026870/Night_At_the_Gates_of_Hell/?snr=1_239_new__145   
Cities: Skylines - Plazas & Promenades          https://store.steampowered.com/app/2008400/Cities_Skylines__Plazas__Promenades/?snr=1_239_new__145   
Lost Judgment                                                         https://store.steampowered.com/app/2058190/Lost_Judgment/?snr=1_239_new__145   
Judgment                                                                   https://store.steampowered.com/app/2058180/Judgment/?snr=1_239_new__145   
Freedom Planet 2                                                    https://store.steampowered.com/app/595500/Freedom_Planet_2/?snr=1_239_new__145   
SCP: Secret Files                                                  https://store.steampowered.com/app/1718130/SCP_Secret_Files/?snr=1_239_new__145   
Isonzo                                                                       https://store.steampowered.com/app/1556790/Isonzo/?snr=1_239_new__145   
Europa Universalis IV: Lions of the North  https://store.steampowered.com/app/2012010/Europa_Universalis_IV_Lions_of_the_North/?snr=1_239_new__145   
Roadwarden                                                               https://store.steampowered.com/app/1155970/Roadwarden/?snr=1_239_new__145   
Taiji                                                                         https://store.steampowered.com/app/1141580/Taiji/?snr=1_239_new__145   
Steelrising                                                             https://store.steampowered.com/app/1283400/Steelrising/?snr=1_239_new__145   

                                           Number of Positive Reviews  \
Name                                                                    
Trombone Champ                                                     31   
hololive ERROR                                                     78   
Absolute Tactics: Daughters of Mercy                               24   
Idle Skilling                                                     241   
Foretales                                                          31   
Metal: Hellsinger                                                1003   
Night At the Gates of Hell                                         55   
Cities: Skylines - Plazas & Promenades                             81   
Lost Judgment                                                     104   
Judgment                                                          402   
Freedom Planet 2                                                  280   
SCP: Secret Files                                                 570   
Isonzo                                                           1407   
Europa Universalis IV: Lions of the North                         219   
Roadwarden                                                        146   
Taiji                                              

In [15]:
games_info.to_csv('final_output.csv', index = True)